In [1]:
from app.Transformer_Classifier import Transformer_Classifier 

from app.Data_Loader import Data_Loader
from app.TextRank_Extractor import TextRank_Extractor
from app.Keyword_Classifier import Keyword_Classifier
from app.common.MySQLUtility import MySQLUtility
import os 
from app.Risk_Score_Service import Risk_Score_Service

domains = ['esg', 'liabilities' ] #'liabilities', 'esg'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/gcp/genuine-wording-key.json'

DB_HOST = '34.170.168.203'
DB_USER = 'root'
DB_PASSWORD = 'nu123456'
DB_NAME = 'lca_dev'

/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Data_ETL_Pipeline(object):
    dbutil = None
    data_load = None
    textrank = None 
    key_classifier = None
    class_service = None
    risk_class = None

    def __init__(self):
        self.dbutil = MySQLUtility(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)
        self.data_load = Data_Loader(self.dbutil)
        self.textrank = TextRank_Extractor(self.dbutil)
        self.key_classifier = Keyword_Classifier(self.dbutil)
        self.class_service = Transformer_Classifier(self.dbutil, domains)
        self.risk_class = Risk_Score_Service(self.dbutil, domains)
        self.risk_class.load_polarity_data()
        pass   

    def keyword_testing(self, text): 
        results = []
        #result['text'] = text
        #c_score = self.risk_class.get_context_score(text, domains[1])
        #result['Context Score'] = c_score
        ''' 
        result = {}
        predict_label, predict_prb = self.key_classifier.predict_text_data(text, domains[1])
        result['type'] = 'keyword'       
        result['label'] = predict_label
        result['score'] = int(predict_prb*100)
        results.append(result)
        '''     
        result = {}
        response2 = self.seed_service.process_contract_request(text, domains[1])
        result['type'] = 'seed'  
        result['label'] = response2[0]['label']
        result['score'] = response2[0]['presence_score']
        results.append(result)

        result = {}
        response1 = self.class_service.process_contract_request(text, domains[1])
        result['type'] = 'transform'  
        result['label'] = response1[0]['label']
        result['score'] = response1[0]['presence_score']
        results.append(result)
        '''
        if response1[0]['label'] == predict_label:
            result['KeyMatch'] = True 
        else: 
            result['KeyMatch'] = False 
        '''
        if response1[0]['label'] == response2[0]['label']:
            result['TransMatch'] = True 
        else: 
            result['TransMatch'] = False 
        
        return results

        


In [3]:
article_text = []
article_text1 = {}
article_text1['name'] = 'Non current Liability'
article_text1['data'] = 'Twenty-five people have been arrested in raids across Germany on suspicion of plotting to overthrow the government. ' + \
        'German reports say the group of far-right and ex-military figures planned to storm the parliament building, the Reichstag, and seize power.' + \
        'A German man referred to as a prince called Heinrich XIII, 71, is alleged to have been central to their plans.' + \
        'Among those arrested in 11 German states were two alleged ringleaders according to federal prosecutors.' 
article_text.append(article_text1)
# current liabilities
article_text1 = {}
article_text1['name'] = 'current liabilities'
article_text1['data'] =  'We can see the company had $6 million in short-term debt for the period. ' + \
        'Accounts payable was broken up into two parts: merchandise payables totaling $1.674 billion, and other accounts payable and accrued liabilities totaling $2.739 billion. ' + \
        'Macy’s had $20 million in taxes payable. ' + \
        'Total liabilities for August 2019 were $4.439 billion, which was nearly unchanged compared to the $4.481 billion for the same accounting period from one year earlier. ' 
article_text.append(article_text1)
# Contengency Liability
article_text1 = {}
article_text1['name'] = 'contingent liabilities'
article_text1['data'] =  'Counter guarantees and guarantees that are given by the company.' + \
        'The company gives a certain guarantee to another stakeholder on behalf of their third party. or it can also be said as the guarantee performed by certain companies as a result of the contract.' + \
        'Product warranty is also given by the company.' + \
        'The company also gives a guarantee on behalf of the stakeholders.' + \
        'The company also issues a letter of credit.' + \
        'The examples also include the adverse judgment of the potential disputes.' 
article_text.append(article_text1)
# Non current Liability 
article_text1 = {}
article_text1['name'] = 'non-current liabilities'
article_text1['data'] = 'Long-term financial liabilities will fall under this category. It may arise from bond payable or bank loans which may be recorded in the balance sheet in the form of amortized cost. ' + \
        'There is a past or current liability which puts a firm under obligation.' + \
        'Liabilities are in the form of borrowing. ' + \
        'When such an obligation is settled, it will lead to a decrease in a firm\'s assets.' + \
        'Deferred tax liability qualifies as a non-current liability. A firm may use a straight-line method of depreciation for financial reporting. Such liability is created when gains or revenue are reflected on the income statement as it becomes eligible to be taxed. ' 
article_text.append(article_text1)

from app.PreProcessText import PreProcessText

etlpipe = Data_ETL_Pipeline()
ppt = PreProcessText()

for article in article_text: 
    print ('Category: ', article['name'])
    stmts = ppt.get_sentences(article['data'])
    for text in stmts: 
        results = etlpipe.keyword_testing(text['sentance'])
        print ('\nInput Text : ', text['sentance'])
        for result in results:
                print (str(result))
    print ('\n')



Category:  Non current Liability

Input Text :  Twenty-five people have been arrested in raids across Germany on suspicion of plotting to overthrow the government.
{'type': 'seed', 'label': 'contingent liabilities', 'score': 97}
{'type': 'transform', 'label': 'contingent liabilities', 'score': 98, 'TransMatch': True}

Input Text :  German reports say the group of far-right and ex-military figures planned to storm the parliament building, the Reichstag, and seize power.
{'type': 'seed', 'label': 'contingent liabilities', 'score': 96}
{'type': 'transform', 'label': 'contingent liabilities', 'score': 95, 'TransMatch': True}

Input Text :  A German man referred to as a prince called Heinrich XIII, , is alleged to have been central to their plans.
{'type': 'seed', 'label': 'contingent liabilities', 'score': 95}
{'type': 'transform', 'label': 'contingent liabilities', 'score': 96, 'TransMatch': True}

Input Text :  Among those arrested in  German states were two alleged ringleaders according

In [4]:
''' 
import spacy
import pytextrank

# example text
text = """Compatibility of systems of linear constraints over the set of natural numbers.
Criteria of compatibility of a system of linear Diophantine equations, strict inequations,
and nonstrict inequations are considered. Upper bounds for components of a minimal set of
solutions and algorithms of construction of minimal generating sets of solutions for all types
of systems are given. These criteria and the corresponding algorithms for constructing a minimal
supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."""

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_md")
# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

doc = nlp(text)
# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)
'''

''' 
article_text.append('Notes and other accounts receivable and other investments are financial assets with carrying values that approximate fair value. ' + \
        'Accounts payable, other accrued expenses and short-term debt (excluding the current portion of long-term debt and including shortterm finance lease liabilities) are financial liabilities with carrying values that approximate fair value. ' + \
        'If measured at fair value in the financial statements, these financial instruments would be classified as Level 3 in the fair value hierarchy, except for short-term debt which would be classified as Level 2. ')
'''

" \narticle_text.append('Notes and other accounts receivable and other investments are financial assets with carrying values that approximate fair value. ' +         'Accounts payable, other accrued expenses and short-term debt (excluding the current portion of long-term debt and including shortterm finance lease liabilities) are financial liabilities with carrying values that approximate fair value. ' +         'If measured at fair value in the financial statements, these financial instruments would be classified as Level 3 in the fair value hierarchy, except for short-term debt which would be classified as Level 2. ')\n"